<a href="https://colab.research.google.com/github/souk2411/lbc/blob/main/Model_Oriententation_Campusen_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.1 MB/s eta 0:00:00


In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.0 MB/s eta 0:00:00


In [4]:
pip install torch

In [5]:
# Importez les bibliothèques nécessaires
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, TrainingArguments, Trainer
import torch
from torch.utils.data import Dataset

# Chargement du tokenizer et le modèle GPT-2 pré-entraîné
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
model = GPT2LMHeadModel.from_pretrained("distilgpt2")

# Définissez un token de padding
tokenizer.pad_token = tokenizer.eos_token  # Vous pouvez utiliser un autre token si nécessaire


In [7]:
# Lire le fichier de données d'orientation
with open("base-entrainement.txt", "r", encoding="utf-8") as file:
    lines = file.read().split("\n")

In [8]:
# Prétraitement des données en paires de questions et de réponses
questions = []
answers = []
for i in range(0, len(lines), 2):
    if i < len(lines) - 1:
        questions.append(lines[i])
        answers.append(lines[i + 1])

# Création d'une classe Dataset personnalisée
class CustomDataset(Dataset):
    def __init__(self, questions, answers, tokenizer, max_length):
        self.questions = questions
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        answer = self.answers[idx]

        encoding = self.tokenizer.encode_plus(
            question,
            answer,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        }

# Utilisation de la classe CustomDataset pour créer votre ensemble de données
max_sequence_length = 512  # Vous pouvez ajuster cette valeur en fonction de vos besoins
custom_dataset = CustomDataset(questions, answers, tokenizer, max_sequence_length)


In [9]:
# Configuration d'entraînement
training_args = TrainingArguments(
    output_dir="./orientation_model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
)

In [10]:
# Création d'un data collator pour le language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)


In [ ]:
# Création d'un objet Trainer et commencement le fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=custom_dataset,  # Utilisation de mon ensemble de données personnalisé
    )
trainer.train()

Step,Training Loss


In [ ]:
# Spécifiez le chemin de sauvegarde pour le modèle
model_save_path = "./le_model"

# Sauvegardez le modèle GPT-2 dans le répertoire spécifié
model.save_pretrained(model_save_path)

# Sauvegardez également le tokenizer
tokenizer.save_pretrained(model_save_path)

In [ ]:
!cp -r le_model/ /content/drive/MyDrive/mon_repertoire_local/

In [ ]:
!mkdir -p /content/drive/MyDrive/mon_repertoire_local/le_model